# This is a combination of all the things

### Downloading dependancies

In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
#from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import string
import time
import datetime
import numpy as np
from cmath import nan

import numpy as np
import pandas as pd
import os

# Word processing libraries
import re
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from shapely.geometry import LineString
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
import folium

### Create Query From Vocab


In [2]:
query_df = pd.read_excel('DATA/Protest-Query.xlsx')

In [3]:
def list_to_or(l):
    out = ''
    for index, word in enumerate(l):
        if word == word:
            out += word + ' OR '

    out = out[:(len(out)-4)]
    return out

In [4]:
uni_kw= list(query_df['University Key Words'])
uni_kw_string = list_to_or(uni_kw)

unis= list(query_df['List of Universities'])
unis_string = list_to_or(unis)

actions= list(query_df['Actions'])
actions_string = list_to_or(actions)

other= list(query_df['Other'])
other_string = list_to_or(other)

In [5]:
query = ('((' + uni_kw_string + ' OR '
            + unis_string + ') ('
            + actions_string + ' OR '
            + other_string + '))')

In [6]:
query += ' -is:retweet lang:en place_country:ZA'

### Creating the Tweet Data. 

In [7]:
# Authenticating Twitter API
# Obtain your Twitter credentials from your twitter developer account

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEGchQEAAAAAZSYFv1nyLDV81YAKEfDr1fVrlho%3DWKBvyLhQ4CeHrlBRtecAetYkB1ZnAjI3Zydb1516fkIzKhS4vh'
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

### Search Criteria

In [8]:
search_words = query
date_since = datetime.datetime(2015,10,1)
date_before = datetime.datetime(2017,12,31)
numTweets = 500

In [9]:
# Scrape Tweets
# Note for the final versions we should use -> until_id -> allos to resume scraping from a particualar tweet ID
protest_tweets = []
count = 0 
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = search_words,
                                 user_fields = ['public_metrics'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 place_fields = ['id', 'geo', 'name', 'country_code', 'place_type', 'full_name', 'country'],
                                 expansions = ['author_id', 'geo.place_id'],
                                 start_time = date_since,
                                 end_time = date_before,
                                 max_results=numTweets
                              ):
   time.sleep(1)
   protest_tweets.append(response)

### Populating Data frame

In [10]:
error_count = 0
result = []
user_dict = {}
place_dict = {}
# Loop through each response object
for response in protest_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                             }
    
    place_dict = {p['id']: p for p in response.includes['places']}

    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet

        try:
            if place_dict[tweet.geo['place_id']]:
                place = place_dict[tweet.geo['place_id']]   
            else:
                place = nan

            result.append({'tweet_id': tweet.id,
                       'author_followers': author_info['followers'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count'],
                       'place_id': tweet.geo,
                       'place_name': place.full_name,
                       'bbox': place.geo['bbox']

                      })

        except:
            error_count = error_count + 1


print(error_count, " result(s) were excluded due to invalid data points.")        

# Change this list of dictionaries into a dataframe             
df = pd.DataFrame(result) 
tweets = df  

981  result(s) were excluded due to invalid data points.


In [11]:
df.to_csv('DATA/temp.csv')

In [12]:
tweets = pd.read_csv('DATA/temp.csv')
tweets = tweets.drop(['Unnamed: 0'], axis=1)
tweets.head(2)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,947179596608364545,1351,@ItIsSeRomeo @maggsnaidu @Julius_S_Malema My q...,2017-12-30 18:56:28+00:00,1,3,11,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396..."
1,947173171156877319,813,@AndileDakuse @eNCA In 1993 was admitted in 3 ...,2017-12-30 18:30:56+00:00,0,0,1,0,{'place_id': 'f16913d3f6acb7f1'},"Mamelodi, South Africa","[28.3213448, -25.7363127, 28.4520433, -25.6853..."


In [13]:
tweets.head(1)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,947179596608364545,1351,@ItIsSeRomeo @maggsnaidu @Julius_S_Malema My q...,2017-12-30 18:56:28+00:00,1,3,11,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396..."


### Adding the previous stuff to it... tweets, location and geolocation

In [14]:
print('Initial size of dataset before dropping duplicated rows:', tweets.shape)
tweets.drop_duplicates(keep = False, inplace = True)

print('Current size of dataset after dropping duplicated rows, if any, is:', tweets.shape)

Initial size of dataset before dropping duplicated rows: (17983, 11)
Current size of dataset after dropping duplicated rows, if any, is: (17983, 11)


In [15]:
tweets.dropna(subset = ['text'], inplace = True)
len(tweets)

17983

### Havent added links and stuff here... We likley wont use in final product. Futhermore text has not been cleaned as this is not needed by either content or sentiment analysis

### Convert Boundry Box to a set of coordinates of Latitude and Longitude
There are a few ways of doing this
* Take an average and find the middle of the Boundry Box
* Indentify where the location is using another API based on place_name
* Keep the location as a polygon and then place each user into a munucipality in hich the area is largest

The way we will do it here is using centroid of the boundry box

### Run from here ======

In [16]:
def bbox_to_coords(bbox):
    all_coords = bbox.split()
    coords = []
    for coordinate in all_coords:
        coordinate = coordinate.replace('[','')
        coordinate = coordinate.replace(']','')
        coordinate = coordinate.replace(',','')
        coord = float(coordinate)
        coords.append(coord)

    return coords

In [17]:
tweets['coords'] = tweets['bbox'].apply(lambda x: bbox_to_coords(x))

In [18]:
tweets['longitude_1'] = tweets['coords'].apply(lambda x: x[0])
tweets['latitude_1'] = tweets['coords'].apply(lambda x: x[1])
tweets['longitude_2'] = tweets['coords'].apply(lambda x: x[2])
tweets['latitude_2'] = tweets['coords'].apply(lambda x: x[3])

In [19]:
tweets = tweets.drop('bbox', axis=1)

In [20]:
def find_centroid(coords):
    geometry = LineString([(coords[0], coords[1]),(coords[2], coords[3])])
    centroid = geometry.centroid
    return centroid

In [21]:
tweets['centroid'] = tweets['coords'].apply(lambda x: find_centroid(x))
tweets['centroid_long'] = tweets['centroid'].apply(lambda z: z.x)
tweets['centroid_lat'] = tweets['centroid'].apply(lambda z: z.y)
tweets = tweets.drop('centroid', axis =1)
tweets.head(3)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,coords,longitude_1,latitude_1,longitude_2,latitude_2,centroid_long,centroid_lat
0,947179596608364545,1351,@ItIsSeRomeo @maggsnaidu @Julius_S_Malema My q...,2017-12-30 18:56:28+00:00,1,3,11,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",27.751856,-26.512649,28.184340,-26.039628,27.968098,-26.276139
1,947173171156877319,813,@AndileDakuse @eNCA In 1993 was admitted in 3 ...,2017-12-30 18:30:56+00:00,0,0,1,0,{'place_id': 'f16913d3f6acb7f1'},"Mamelodi, South Africa","[28.3213448, -25.7363127, 28.4520433, -25.6853...",28.321345,-25.736313,28.452043,-25.685311,28.386694,-25.710812
2,947172303732912129,9119,@UliMahlangu President Zuma stole the thunder...,2017-12-30 18:27:29+00:00,2,2,3,0,{'place_id': '8b9ec16fdc0d7e55'},"Cape Town, South Africa","[18.3180332, -34.35839, 18.6600898, -33.8849254]",18.318033,-34.358390,18.660090,-33.884925,18.489061,-34.121658


### Fix Edge Cases
* Cape Town
* Betty's Bay
* Bloubergstrand
* Mdumbi Beach 
</br>
Using: https://www.distancesto.com/coordinates/za/bloubergstrand-latitude-longitude/history/76385.html

In [22]:
tweets.loc[tweets.place_name == 'Cape Town, South Africa', ['centroid_long', 'centroid_lat']] = 18.4241, -33.9249
tweets.loc[tweets.place_name == 'Mdumbi Beach', ['centroid_long', 'centroid_lat']] = 29.215369, -31.933896
tweets.loc[tweets.place_name == "Betty's Bay, South Africa", ['centroid_long', 'centroid_lat']] = 18.92051, -34.34747
tweets.loc[tweets.place_name == 'Bloubergstrand', ['centroid_long', 'centroid_lat']] = 18.46173, -33.800418

In [23]:
geometry = [Point(xy) for xy in zip(tweets['centroid_long'], tweets['centroid_lat'])]
gdf = GeoDataFrame(tweets, geometry=geometry) 

In [24]:
tweets = tweets.drop('longitude_1', axis=1)
tweets = tweets.drop('longitude_2', axis=1)
tweets = tweets.drop('latitude_1', axis=1)
tweets = tweets.drop('latitude_2', axis=1)
tweets = tweets.drop('centroid_long', axis=1)
tweets = tweets.drop('centroid_lat', axis=1)
tweets.head(2)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,coords,geometry
0,947179596608364545,1351,@ItIsSeRomeo @maggsnaidu @Julius_S_Malema My q...,2017-12-30 18:56:28+00:00,1,3,11,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",POINT (27.96810 -26.27614)
1,947173171156877319,813,@AndileDakuse @eNCA In 1993 was admitted in 3 ...,2017-12-30 18:30:56+00:00,0,0,1,0,{'place_id': 'f16913d3f6acb7f1'},"Mamelodi, South Africa","[28.3213448, -25.7363127, 28.4520433, -25.6853...",POINT (28.38669 -25.71081)


## Removing Useless locations

In [25]:
tweets = tweets[tweets['place_name'] != 'South Africa']
tweets.head(2)

,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,coords,geometry
0,947179596608364545,1351,@ItIsSeRomeo @maggsnaidu @Julius_S_Malema My q...,2017-12-30 18:56:28+00:00,1,3,11,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",POINT (27.96810 -26.27614)
1,947173171156877319,813,@AndileDakuse @eNCA In 1993 was admitted in 3 ...,2017-12-30 18:30:56+00:00,0,0,1,0,{'place_id': 'f16913d3f6acb7f1'},"Mamelodi, South Africa","[28.3213448, -25.7363127, 28.4520433, -25.6853...",POINT (28.38669 -25.71081)


# Adding location based on shape file

In [26]:
#Nan
from cmath import nan


import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
from shapely.geometry import LineString
from sympy import centroid
from geopandas import GeoDataFrame
# importing geopy library
from geopy.geocoders import Nominatim
from numpy import NaN
from shapely import wkt

# importing all the dependancies
import numpy as np
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from datetime import datetime
from datetime import timedelta

import pandas as pd
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

### Downloading Tweets

### Converting created at to a datetime object

In [27]:
from datetime import datetime

dates = tweets['created_at']
datesOBJ =[]
for date in dates:
    date_object = datetime.fromisoformat(date)
    date_object = date_object.date()
    datesOBJ.append(date_object)

tweets['Date'] = datesOBJ

### Downloading Tweets

In [28]:
tweets = tweets[['text','Date','geometry','author_followers','retweets','replies','likes','quote_count']]

In [29]:
tweets.head(2)

,text,Date,geometry,author_followers,retweets,replies,likes,quote_count
0,@ItIsSeRomeo @maggsnaidu @Julius_S_Malema My q...,2017-12-30,POINT (27.96810 -26.27614),1351,1,3,11,0
1,@AndileDakuse @eNCA In 1993 was admitted in 3 ...,2017-12-30,POINT (28.38669 -25.71081),813,0,0,1,0


# Adding the sentiment

It would be way better if i made this a pandas function as opposed to creating a separte list. 

In [30]:
# creating a dataframe with the text as the list
text = tweets['text'].tolist()
text = pd.DataFrame(text, columns = ['Text'])

In [31]:
# this is a cleaning text function... very simular to the one in tweet_cleaning. May just combine this into one function in tweet cleaning. 
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # removes mentions
    text = re.sub(r'#', '', text) # removes #
    text = re.sub(r'RT[\s]+', '', text) # removes RT
    text = re.sub(r'https?:\/\/\S+', '', text) # removes hyper link

    return text
text['Text']=text['Text'].apply(cleanTxt)

In [32]:
# get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
    
text['Subjectivity'] = text['Text'].apply(getSubjectivity)
text['Polarity'] = text['Text'].apply(getPolarity)
text['Analysis'] = text['Polarity'].apply(getAnalysis)


text = text.drop(['Text'], axis = 1)

In [33]:
# Adding the subjectivity to the tweets
tweets = pd.concat([tweets, text.reindex(tweets.index)], axis = 1)

# Content Analysis


### Define String Extractor

In [34]:
def string_substring(string, substring):
    # Lower ensures that the vocab is case insensitive
    string = string.lower()
    substring = substring.lower()

    if substring in string:
        return True
    else:
        return False

### Content Analysis method
Takes in a vocabulary as argument and outputs a list that corresponds to a list of concepts for each tweet

In [35]:
def content_analysis(vocabulary):
    list_ = []

    # Loop Through Tweets
    for iter_t, tweet in tweets.iterrows():
        sub_list = []
        #Loop Through Vocab
        for iter_g, vocab in vocabulary.iterrows():
            # Loop Through Phrases in Vocab
            for phrase in vocab['Phrases']:
                # Check if Phrase is in Text
                if string_substring(tweet['text'], phrase):
                    if vocab['Conditional_Phrases'] != vocab['Conditional_Phrases']:
                        sub_list.append(vocab['Concept'])
                        break
                    else:
                        # Loop Through Conditional Phrases in Vocab
                        for con_phrase in vocab['Conditional_Phrases']:
                            # Check to see if there are Conditional Phrases otherwise break
                            if string_substring(tweet['text'], con_phrase):
                                sub_list.append(vocab['Concept'])
                                break
                            # Check if the conditional phrase has a NOT Operator
                            elif '%not%' in con_phrase:
                                con_phrase = con_phrase[6:]
                                if not string_substring(tweet['text'], con_phrase):
                                    sub_list.append(vocab['Concept'])
                                    break
        if len(sub_list) != 0:
            list_.append(sub_list)
        else:
            list_.append(nan)
    return list_

### Load in All Vocabs
And ensure all vocab lists set up correctly

In [36]:
def split_string(string):
    if string == string:
        string = string.lower()
        string = string.replace("'", "")
        out = string.strip('][').split(', ')
        return out
    else:
        return nan

In [37]:
grievances = pd.read_excel('DATA/Vocabularies/grievances.xlsx')
grievances.drop('Unnamed: 0', axis = 1, inplace = True)
grievances['Phrases'] = grievances['Phrases'].apply(lambda x: split_string(x))
grievances['Conditional_Phrases'] = grievances['Conditional_Phrases'].apply(lambda x: split_string(x))

In [38]:
triggers = pd.read_excel('DATA/Vocabularies/trigger.xlsx')
triggers.drop('Unnamed: 0', axis = 1, inplace = True)
triggers['Phrases'] = triggers['Phrases'].apply(lambda x: split_string(x))
triggers['Conditional_Phrases'] = triggers['Conditional_Phrases'].apply(lambda x: split_string(x))

In [39]:
tactics = pd.read_excel('DATA/Vocabularies/tactic.xlsx')
tactics.drop('Unnamed: 0', axis = 1, inplace = True)
tactics['Phrases'] = tactics['Phrases'].apply(lambda x: split_string(x))
tactics['Conditional_Phrases'] = tactics['Conditional_Phrases'].apply(lambda x: split_string(x))

In [40]:
actors = pd.read_excel('DATA/Vocabularies/actors.xlsx')
actors.drop('Unnamed: 0', axis = 1, inplace = True)
actors['Phrases'] = actors['Phrases'].apply(lambda x: split_string(x))
actors['Conditional_Phrases'] = actors['Conditional_Phrases'].apply(lambda x: split_string(x))

In [41]:
locations = pd.read_excel('DATA/Vocabularies/locations.xlsx')
locations.drop('Unnamed: 0', axis = 1, inplace = True)
locations['Phrases'] = locations['Phrases'].apply(lambda x: split_string(x))
locations['Conditional_Phrases'] = locations['Conditional_Phrases'].apply(lambda x: split_string(x))

In [42]:
weapons = pd.read_excel('DATA/Vocabularies/weapons.xlsx')
weapons.drop('Unnamed: 0', axis = 1, inplace = True)
weapons['Phrases'] = weapons['Phrases'].apply(lambda x: split_string(x))
weapons['Conditional_Phrases'] = weapons['Conditional_Phrases'].apply(lambda x: split_string(x))

In [43]:
eventualities = pd.read_excel('DATA/Vocabularies/eventuality.xlsx')
eventualities.drop('Unnamed: 0', axis = 1, inplace = True)
eventualities['Phrases'] = eventualities['Phrases'].apply(lambda x: split_string(x))
eventualities['Conditional_Phrases'] = eventualities['Conditional_Phrases'].apply(lambda x: split_string(x))

In [44]:
curiosities = pd.read_excel('DATA/Vocabularies/curiosity.xlsx')
curiosities.drop('Unnamed: 0', axis = 1, inplace = True)
curiosities['Phrases'] = curiosities['Phrases'].apply(lambda x: split_string(x))
curiosities['Conditional_Phrases'] = curiosities['Conditional_Phrases'].apply(lambda x: split_string(x))

In [45]:
non_protests = pd.read_excel('DATA/Vocabularies/non_protest.xlsx')
non_protests.drop('Unnamed: 0', axis = 1, inplace = True)
non_protests['Phrases'] = non_protests['Phrases'].apply(lambda x: split_string(x))
non_protests['Conditional_Phrases'] = non_protests['Conditional_Phrases'].apply(lambda x: split_string(x))

###  Content Analysis for all vocabs:
* Grievances
* Trigger
* Tactic
* Actors
* Location
* Weapons
* Eventuality
* Curiosity
* Non-Protest

</br>
And Remove any duplicates

In [46]:
def unique_list(x):
    if x == x:
        return list(set(x))
    else:
        return nan

In [47]:
tweets['grievances'] = content_analysis(grievances)
tweets['grievances'] = tweets['grievances'].apply(lambda x: unique_list(x))

In [48]:
tweets['triggers'] = content_analysis(triggers)
tweets['triggers'] = tweets['triggers'].apply(lambda x: unique_list(x))

In [49]:
tweets['tactics'] = content_analysis(tactics)
tweets['tactics'] = tweets['tactics'].apply(lambda x: unique_list(x))

In [50]:
tweets['actors'] = content_analysis(actors)
tweets['actors'] = tweets['actors'].apply(lambda x: unique_list(x))

In [51]:
tweets['locations'] = content_analysis(locations)
tweets['locations'] = tweets['locations'].apply(lambda x: unique_list(x))

In [52]:
tweets['weapons'] = content_analysis(weapons)
tweets['weapons'] = tweets['weapons'].apply(lambda x: unique_list(x))

In [53]:
tweets['eventualities'] = content_analysis(eventualities)
tweets['eventualities'] = tweets['eventualities'].apply(lambda x: unique_list(x))

In [54]:
tweets['curiosities'] = content_analysis(curiosities)
tweets['curiosities'] = tweets['curiosities'].apply(lambda x: unique_list(x))

In [55]:
tweets['non_protests'] = content_analysis(non_protests)
tweets['non_protests'] = tweets['non_protests'].apply(lambda x: unique_list(x))

### Content and Frequency Tables

In [56]:
content_table = pd.read_excel('DATA/Protest is SA   SAS rules V2 - Separated.xlsx')
content_table = content_table.drop('Phrases', axis =1)
content_table = content_table.drop('Conditional_Phrases', axis =1)
content_table = content_table.drop('Afrikaans_Phrases', axis =1)
content_table = content_table.drop('Rule', axis =1)
content_table['Occurances'] = 0

In [57]:
def count_occurances(col_name, label):
    global content_table
    occurances = list(content_table['Occurances'])
    for iter_t, tweet in tweets.iterrows():
            if tweet[col_name] == tweet[col_name]:
                for content in tweet[col_name]:
                    for iter_c, concept in content_table.iterrows():
                        if concept['Concept'] == content:
                            if concept['Label'] == label:
                                occurances[iter_c] += 1
                            
    content_table = content_table.drop('Occurances', axis =1)
    content_table['Occurances'] = occurances

In [58]:
count_occurances('grievances', 'Grievance')
print('Counted Grievances!')
count_occurances('triggers', 'Trigger')
print('Counted Triggers!')
count_occurances('tactics', 'Tactic')
print('Counted Tactics!')
count_occurances('actors', 'Actors')
print('Counted Actors!')
count_occurances('locations', 'Location')
print('Counted Locations!')
count_occurances('weapons', 'Weapons')
print('Counted Weapons!')
count_occurances('eventualities', 'Eventuality')
print('Counted Eventualities!')
count_occurances('curiosities', 'Curiosity')
print('Counted Curiosities!')
count_occurances('non_protests', 'Non-protest')
print('Counted Non-Protests!')
print('Done!')

Counted Grievances!
Counted Triggers!
Counted Tactics!
Counted Actors!
Counted Locations!
Counted Weapons!
Counted Eventualities!
Counted Curiosities!
Counted Non-Protests!
Done!


In [59]:
content_table.to_csv('DATA/Content Table.csv')

In [60]:
total = len(tweets)
frequency_table = content_table.groupby('Label').Occurances.sum().reset_index()
frequency_table['Frequency'] = frequency_table['Occurances'].apply(lambda x: round((x/total)*100,2))

In [61]:
frequency_table.to_csv('DATA/Frequency Content Table.csv')

# University Locator

In [62]:
universities = pd.read_excel('DATA/South African Universities.xlsx')

In [63]:
def string_substring_case_sensitive(string, substring):
    # Lower ensures that the vocab is case insensitive
    if substring in string:
        return True
    else:
        return False

In [64]:
universities['Nickname'] = universities['Nickname'].apply(lambda x: split_string(x))
universities['Abbriviation'] = universities['Abbriviation'].apply(lambda x: split_string(x))

In [65]:
def university_locator():
    addition = []
    for iter_t, tweet in tweets.iterrows():
        coords = []
        words = []
        for iter_u, uni in universities.iterrows():
            if uni['Nickname'] == uni['Nickname']:
                for nickname in uni['Nickname']:
                    if string_substring(tweet['text'], nickname):
                        # coords = [uni['Latitude'], uni['Longitude']]#lat, long
                        words.append(uni['Universities'])
            if uni['Abbriviation'] == uni['Abbriviation']:        
                for abbr in uni['Abbriviation']:
                    if string_substring_case_sensitive(tweet['text'], abbr):
                        words.append(uni['Universities'])
                
                if string_substring(tweet['text'], uni['Universities']):
                    words.append(uni['Universities'])
        words = list(set(words))
        if len(words) == 0:
            words = nan
        addition.append(words)

    return addition

In [66]:
tweets['universities'] = university_locator()

In [67]:
def change_coords():
    for iter_t, tweet in tweets.iterrows():
        if tweet['universities'] == tweet['universities']:
            if len(tweet['universities']) == 1:
                new_lat = (list(universities.loc[universities['Universities'] == tweet['universities'][0], 'Latitude']))[0]
                new_long = (list(universities.loc[universities['Universities'] == tweet['universities'][0], 'Longitude']))[0]
                new_geometry = Point(new_long,new_lat)
                tweets['geometry'][iter_t] = new_geometry

In [68]:
change_coords()

In [69]:
tweets.shape

(16843, 21)

In [70]:

tweets.to_csv('DATA/rawtweets.csv')